In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import keras
import numpy as np
import os
from six.moves import xrange

import tensorflow as tf
from tensorflow.python.platform import app
from tensorflow.python.platform import flags

from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval, batch_eval

from cleverhans.attacks import fgsm
from cleverhans.attacks_tf import jacobian_graph
from cleverhans.utils import other_classes, cnn_model, pair_visual, grid_visual
from Gnet import G_10,G_11,G_12,G_13




In [4]:
FLAGS = flags.FLAGS

flags.DEFINE_string('train_dir', '/home/boyuan/Documents/research/Advesarial_examples/Project/test_store/test1', 'Directory storing the saved model.')
flags.DEFINE_string('filename', 'cnnmodel.ckpt', 'Filename to save model under.')
flags.DEFINE_integer('nb_epochs', 6, 'Number of epochs to train model')
flags.DEFINE_integer('batch_size', 128, 'Size of training batches')
flags.DEFINE_float('learning_rate', 0.1, 'Learning rate for training')

In [5]:
import tensorflow as tf
# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)

###########################################################################
# Define the dataset and model
###########################################################################

# Image dimensions ordering should follow the Theano convention
if keras.backend.image_dim_ordering() != 'tf':
    keras.backend.set_image_dim_ordering('tf')
    print("INFO: '~/.keras/keras.json' sets 'image_dim_ordering' "
          "to 'th', temporarily setting to 'tf'")

# Create TF session and set as Keras backend session
sess = tf.Session()
keras.backend.set_session(sess)
print("Created TensorFlow session and set Keras backend.")

# Get MNIST test data
X_train, Y_train, X_test, Y_test = data_mnist()
print("Loaded MNIST test data.")

# Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y = tf.placeholder(tf.float32, shape=(None, 10))
###########################################################################
# Training the model using TensorFlow
###########################################################################
save_path = os.path.join(FLAGS.train_dir, FLAGS.filename)
# Define TF model graph
model = G_11()
predictions_1 = model(x)

print("Defined TensorFlow model graph.")
saver = tf.train.Saver()

save_path2 = '/home/boyuan/Documents/research/Advesarial_examples/Project/store/G11/G11model'
#save_path3 = '/home/boyuan/Documents/research/Advesarial_examples/Project/store/G10/G10model'
saver.restore(sess, save_path2)
#saver.restore(sess, save_path3)
# Evaluate the accuracy of the MNIST model on legitimate test examples
eval_params = {'batch_size': FLAGS.batch_size}
accuracy = model_eval(sess, x, y, predictions_1, X_test, Y_test,
                      args=eval_params)
assert X_test.shape[0] == 10000, X_test.shape
print('Test accuracy on legitimate test examples: {0}'.format(accuracy))


Created TensorFlow session and set Keras backend.
X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Loaded MNIST test data.
Defined TensorFlow model graph.
INFO:tensorflow:Restoring parameters from /home/boyuan/Documents/research/Advesarial_examples/Project/store/G11/G11model
Test accuracy on legitimate test examples: 0.9849


In [14]:
# Craft adversarial examples using Fast Gradient Sign Method (FGSM)
adv_x = fgsm(x, predictions_1, eps=0.3)
eval_params = {'batch_size': FLAGS.batch_size}
X_test_adv, = batch_eval(sess, [x], [adv_x], [X_test], args=eval_params)
assert X_test_adv.shape[0] == 10000, X_test_adv.shape



NameError: name 'predictions' is not defined

In [15]:
# Evaluate the accuracy of the MNIST model on adversarial examples
accuracy = model_eval(sess, x, y, predictions_1, X_test_adv, Y_test,
                      args=eval_params)
print('Test accuracy on adversarial examples: ' + str(accuracy))

Test accuracy on adversarial examples: 0.1279


In [22]:
X_test[0].shape[0]

28

In [24]:
print("Repeating the process, using adversarial training")
# Redefine TF model graph
model_2 = G_11()
predictions_2 = model_2(x)
adv_x_2 = fgsm(x, predictions_2, eps=0.3)
predictions_2_adv = model_2(adv_x_2)

def evaluate_2():
    # Evaluate the accuracy of the adversarialy trained MNIST model on
    # legitimate test examples
    eval_params = {'batch_size': FLAGS.batch_size}
    accuracy = model_eval(sess, x, y, predictions_2, X_test, Y_test,
                          args=eval_params)
    print('Test accuracy on legitimate test examples: ' + str(accuracy))

    # Evaluate the accuracy of the adversarially trained MNIST model on
    # adversarial examples
    accuracy_adv = model_eval(sess, x, y, predictions_2_adv, X_test,
                              Y_test, args=eval_params)
    print('Test accuracy on adversarial examples: ' + str(accuracy_adv))

save_path3 = '/home/boyuan/Documents/research/Advesarial_examples/Project/store/G11_adv'
# Perform adversarial training
train_params = {
    'nb_epochs': FLAGS.nb_epochs,
    'batch_size': FLAGS.batch_size,
    'learning_rate': FLAGS.learning_rate,
    'train_dir': save_path3,
    'filename': 'G11_advmodel'
}
 
model_train(sess, x, y, predictions_2, X_train, Y_train, save = True,
            predictions_adv=predictions_2_adv, evaluate=evaluate_2,
            args=train_params)

Repeating the process, using adversarial training
Epoch 0
	Epoch took 2597.62343216 seconds
Test accuracy on legitimate test examples: 0.9475
Test accuracy on adversarial examples: 0.5444
Epoch 1
	Epoch took 2630.78867388 seconds
Test accuracy on legitimate test examples: 0.9654
Test accuracy on adversarial examples: 0.5825
Epoch 2
	Epoch took 2795.96334314 seconds
Test accuracy on legitimate test examples: 0.9718
Test accuracy on adversarial examples: 0.6261
Epoch 3
	Epoch took 2692.30176401 seconds
Test accuracy on legitimate test examples: 0.975
Test accuracy on adversarial examples: 0.5941
Epoch 4
	Epoch took 2604.72439909 seconds
Test accuracy on legitimate test examples: 0.9776
Test accuracy on adversarial examples: 0.8205
Epoch 5
	Epoch took 2552.36292696 seconds
Test accuracy on legitimate test examples: 0.9807
Test accuracy on adversarial examples: 0.7155
Completed model training and saved at:/home/boyuan/Documents/research/Advesarial_examples/Project/store/G11_adv/G11_advmode

True